In [1]:
import os
import joblib
import numpy as np

# gdown 라이브러리 설치 (코랩에 없을 경우 대비)
!pip install -q gdown

print("✅ 라이브러리 로드 완료")

# -------------------------------------------------------------------
# 📥 모델 가중치 자동 다운로드 (Google Drive)
# -------------------------------------------------------------------
# ⚠️ [중요] 아까 복사한 본인의 File ID를 아래 따옴표 안에 넣으세요!
file_id = '138_2NXbeGm2wCCaOVeDd12c7hGrm7alE'
destination = 'news_classifier_model.pkl'

def download_file_from_google_drive(id, destination):
    import gdown
    url = f'https://drive.google.com/uc?id={id}'
    # quiet=False로 설정하여 다운로드 과정을 보여줌
    gdown.download(url, destination, quiet=False)

# 파일이 현재 경로에 없으면 다운로드 실행
if not os.path.exists(destination):
    print(f"📥 구글 드라이브에서 모델 파일({destination})을 다운로드합니다...")
    try:
        download_file_from_google_drive(file_id, destination)
        print("✅ 다운로드 및 저장 완료!")
    except Exception as e:
        print(f"❌ 다운로드 실패: {e}")
        print("   File ID가 정확한지, 링크 공유 권한이 '링크가 있는 모든 사용자'인지 확인해주세요.")
else:
    print(f"✅ 모델 파일({destination})이 이미 존재합니다. (다운로드 건너뜀)")

✅ 라이브러리 로드 완료
📥 구글 드라이브에서 모델 파일(news_classifier_model.pkl)을 다운로드합니다...


Downloading...
From: https://drive.google.com/uc?id=138_2NXbeGm2wCCaOVeDd12c7hGrm7alE
To: /content/news_classifier_model.pkl
100%|██████████| 4.08M/4.08M [00:00<00:00, 14.7MB/s]

✅ 다운로드 및 저장 완료!


In [4]:
model_file = 'news_classifier_model.pkl'

try:
    # 1. 딕셔너리 객체 로드
    saved_object = joblib.load(model_file)

    # 2. 모델과 인코더 분리
    model = saved_object['model']   # 학습된 파이프라인 (TF-IDF + MLP)
    le = saved_object['encoder']    # 라벨 인코더 (숫자 <-> 카테고리 변환기)

    print("🧠 모델 로드 성공!")
    print(f"   - 학습된 클래스: {le.classes_}")

except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("   파일이 손상되었거나, training 코드 버전이 맞지 않을 수 있습니다.")

🧠 모델 로드 성공!
   - 학습된 클래스: ['CS_Insight' 'Economy' 'Politics' 'Society' 'Technology']


In [5]:
def predict_news(headline):
    """
    뉴스 제목을 입력받아 예측 카테고리와 확률을 반환하는 함수
    """
    # 1. 모델 예측 (입력은 리스트 형태여야 함)
    # MLP는 확률(proba)을 계산할 수 있음
    pred_proba = model.predict_proba([headline])

    # 2. 가장 높은 확률의 인덱스 찾기
    max_idx = np.argmax(pred_proba)
    confidence = pred_proba[0][max_idx] # 확신도 (0~1)

    # 3. 숫자를 다시 문자로 변환
    pred_label = le.classes_[max_idx]

    return pred_label, confidence

print("✅ 추론 함수 준비 완료")

✅ 추론 함수 준비 완료


In [6]:
# 테스트할 실제 뉴스 헤드라인들
examples = [
    # [Case 1] 명확한 CS 뉴스 (우리의 타겟!)
    "NVIDIA announces new Blackwell AI chip for data centers",

    # [Case 2] 명확한 경제 뉴스
    "Federal Reserve raises interest rates to fight inflation",

    # [Case 3] 명확한 스포츠 뉴스
    "Premier League: Liverpool beats Chelsea in a thrilling match",

    # [Case 4] 애매한 것 (기술 + 경제) -> 과연 모델은?
    "Tech stocks tumble as Apple and Google face regulatory pressure"
]

print("🔍 [실전 테스트 결과]\n")

for i, headline in enumerate(examples):
    label, conf = predict_news(headline)

    print(f"📰 뉴스 {i+1}: \"{headline}\"")
    print(f"➡️ 예측 결과: [{label}] (확신도: {conf:.2%})")
    print("-" * 50)

🔍 [실전 테스트 결과]

📰 뉴스 1: "NVIDIA announces new Blackwell AI chip for data centers"
➡️ 예측 결과: [CS_Insight] (확신도: 86.86%)
--------------------------------------------------
📰 뉴스 2: "Federal Reserve raises interest rates to fight inflation"
➡️ 예측 결과: [Society] (확신도: 66.66%)
--------------------------------------------------
📰 뉴스 3: "Premier League: Liverpool beats Chelsea in a thrilling match"
➡️ 예측 결과: [Society] (확신도: 29.17%)
--------------------------------------------------
📰 뉴스 4: "Tech stocks tumble as Apple and Google face regulatory pressure"
➡️ 예측 결과: [CS_Insight] (확신도: 72.48%)
--------------------------------------------------


In [7]:
print("✍️ 직접 뉴스 제목을 입력해보세요! (종료하려면 'q' 입력)")

while True:
    user_input = input("\n뉴스 헤드라인 입력: ")

    if user_input.lower() == 'q':
        print("종료합니다.")
        break

    if not user_input.strip():
        continue

    label, conf = predict_news(user_input)
    print(f"🤖 모델의 판단: [{label}] (확신도: {conf:.2%})")

✍️ 직접 뉴스 제목을 입력해보세요! (종료하려면 'q' 입력)

뉴스 헤드라인 입력: Trump Administration Live Updates: President Calls Somalis ‘Garbage’ in Anti-Immigrant Tirade
🤖 모델의 판단: [Politics] (확신도: 32.43%)

뉴스 헤드라인 입력: q
종료합니다.
